<a href="https://colab.research.google.com/github/earo12/Deploy_Consultancy/blob/dev-branch/Automatizacion_consultora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [4]:
# Conexión a la BD:

def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='admin',
            database='consultora_prueba'
        )
        if connection.is_connected():
            return connection
    except Error as e:
        print(f"Error al conectar con MySQL: {e}")
        return None
